In [1]:
import cv2 as cv
import numpy as np

In [4]:
!pip freeze > requirements.txt

In [7]:
def mask_region_of_interest(image, vertices):
    # Cria uma máscara preta com as mesmas dimensões que a imagem de entrada
    mask = np.zeros_like(image)
    
    # Define a cor para preencher a região de interesse (geralmente branca)
    match_mask_color = 255
    
    # Preenche a região de interesse definida pelos vértices com a cor especificada
    cv.fillPoly(mask, vertices, match_mask_color)
    
    # Aplica a máscara à imagem original usando a operação AND bit a bit
    masked_image = cv.bitwise_and(image, mask)
    
    # Retorna a imagem resultante, onde apenas a região de interesse é preservada
    return masked_image

In [61]:
def draw_the_lines(image, lines):
    # Faz uma cópia da imagem original para não modificar a original
    image = np.copy(image)
    
    # Cria uma imagem em branco do mesmo tamanho que a imagem original
    blank_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    
    # Verifica se existem linhas para desenhar
    if lines is not None:
        # Para cada linha detectada
        for line in lines:
            # Extrai as coordenadas dos pontos inicial e final da linha
            for x1, y1, x2, y2 in line:
                # Desenha a linha na imagem em branco
                cv.line(blank_image, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=3)
    
    # Combina as linhas desenhadas na imagem em branco com a imagem original usando uma ponderação
    # A imagem original é mais pesada (0.8) do que a imagem em branco (1)
    # Isso produz um efeito de transparência para as linhas desenhadas
    image = cv.addWeighted(image, 0.8, blank_image, 1, 0.0)
    
    # Retorna a imagem resultante com as linhas desenhadas
    return image

In [72]:
# Abrir o vídeo
cap = cv.VideoCapture("estrada_imagem.mp4")

# Obter a taxa de quadros do vídeo original
fps = cap.get(cv.CAP_PROP_FPS)

while True:
    # Capturar o próximo frame
    ret, frame = cap.read()
    
    # Verificar se o frame foi capturado corretamente
    if not ret:
        break

    # Redimensionar o frame para 720p
    frame = cv.resize(frame, (1280, 720))

    # Exibir o frame
    #cv.imshow('estrada', frame)
    
    h, w = frame.shape[:2]

    # Definir a região de interesse como um triângulo na parte inferior do quadro
  
    region_of_interest = np.array([[
        (0, h), # Inf Esquerdo
        (w, h), # Inf direito
        (w*.15, h * 2 // 3), # sup esquerdo
        (w*.25, h * 2 // 3)]], dtype=np.int32) # Sup direito
    
    # Converter o quadro para escala de cinza e aplicar detecção de bordas
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    frame_canny = cv.Canny(frame_gray, 100, 150)

    # Aplicar a máscara para manter apenas a região de interesse
    frame_masked = mask_region_of_interest(frame_canny, region_of_interest)
    #cv.imshow('Video com Linhas', frame_masked)

    
    # Detectar linhas usando a Transformada de Hough Probabilística
    lines = cv.HoughLinesP(
        frame_masked,
        rho=1,
        theta=np.pi/50,
        threshold=5,
        minLineLength=5,
        maxLineGap=15
    )

    # Desenhar as linhas no quadro original
    frame_with_lines = draw_the_lines(frame, lines)

    # Exibir o quadro com as linhas detectadas
    cv.imshow('Video com Linhas', frame_with_lines)
    
    # Aguardar pela tecla de saída (ESC)
    key = cv.waitKey(int(500 / fps))  # Ajustar o tempo de espera para corresponder ao FPS do vídeo
    if key == 27:
        break

# Liberar os recursos
cap.release()
cv.destroyAllWindows()